In [8]:
import glob
import os 
import numpy as np
import cv2
import mediapipe
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import keras 
import keras_tuner
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.utils import np_utils
import tensorflow.keras
from keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split

In [2]:
names = ['buildings','forest', 'glacier','mountain', 'sea', 'street']

In [19]:
train_data = []
train_image_names=[]
train_labels = []
test_data = []
test_labels = []
k=0
for i in names:
    print(i)
    os.chdir(r'/Users/chandu/Downloads/Intel Image Classification/seg_train/seg_train/'+i+'/')
    file_name = os.listdir()
    train_image_names = train_image_names + file_name
    for j in file_name:
        img=cv2.imread(r'/Users/chandu/Downloads/Intel Image Classification/seg_train/seg_train/'+i+'/'+j)
        img = cv2.resize(img,(100,100))
        train_data.append(img)
        train_labels.append(k)
    os.chdir(r'/Users/chandu/Downloads/Intel Image Classification/seg_test/seg_test/'+i+'/')
    file_name = os.listdir()
    for j in file_name:
        img=cv2.imread(r'/Users/chandu/Downloads/Intel Image Classification/seg_test/seg_test/'+i+'/'+j)
        img = cv2.resize(img,(100,100))
        test_data.append(img)
        test_labels.append(k) 
    k=k+1

buildings
forest
glacier
mountain
sea
street


In [20]:
train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [21]:
train_data = train_data/255
test_data = test_data/255

In [22]:
train_label = keras.utils.to_categorical(train_labels,dtype='float32')
test_label = keras.utils.to_categorical(test_labels,dtype='float32')

In [9]:
x_train, x_test, y_train, y_test = train_test_split(train_data,train_label,stratify=train_label,test_size=0.08,random_state=100)

In [10]:
main=VGG16(include_top=False,weights='imagenet',input_shape=(100,100,3))

2023-04-19 00:50:53.895148: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
main.trainable=False

In [12]:
model=Sequential()
model.add(main)
model.add(Flatten())
model.add(Dense(128,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(6,activation='softmax'))

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 128)               589952    
                                                                 
 dense_1 (Dense)             (None, 6)                 774       
                                                                 
Total params: 15,305,414
Trainable params: 590,726
Non-trainable params: 14,714,688
_________________________________________________________________


In [14]:
model.fit(x_train,y_train,batch_size=32,epochs=5,validation_split=0.20)

Epoch 1/5
323/323 [==============================] - 1263s 4s/step - loss: 0.4860 - accuracy: 0.8219 - val_loss: 0.5100 - val_accuracy: 0.8258
Epoch 2/5
323/323 [==============================] - 1273s 4s/step - loss: 0.3205 - accuracy: 0.8842 - val_loss: 0.4299 - val_accuracy: 0.8482
Epoch 3/5
323/323 [==============================] - 1259s 4s/step - loss: 0.2581 - accuracy: 0.9082 - val_loss: 0.3761 - val_accuracy: 0.8695
Epoch 4/5
323/323 [==============================] - 1256s 4s/step - loss: 0.2207 - accuracy: 0.9169 - val_loss: 0.4498 - val_accuracy: 0.8494
Epoch 5/5
323/323 [==============================] - 1368s 4s/step - loss: 0.1915 - accuracy: 0.9313 - val_loss: 0.3956 - val_accuracy: 0.8691


In [15]:
model.evaluate(x_test,y_test)

36/36 [==============================] - 120s 3s/step - loss: 0.4155 - accuracy: 0.8629


[0.41547420620918274, 0.8628672957420349]

In [16]:
y_train_pred = model.predict(train_data)
y_train_pred

439/439 [==============================] - 1493s 3s/step


array([[9.0356332e-01, 3.7617497e-03, 6.3410148e-02, 9.6321088e-04,
        5.4744673e-03, 2.2827022e-02],
       [9.9999976e-01, 4.3209490e-13, 1.2857336e-07, 3.4587875e-09,
        1.3289987e-07, 4.4925556e-12],
       [9.9809152e-01, 1.1808969e-07, 3.0800340e-06, 1.0927095e-07,
        8.3854084e-06, 1.8967028e-03],
       ...,
       [8.0357008e-02, 1.9014433e-08, 5.3638560e-05, 2.9117550e-06,
        3.1459865e-06, 9.1958332e-01],
       [1.8389991e-02, 3.2299390e-08, 4.2773976e-05, 1.7765377e-07,
        3.9744205e-06, 9.8156309e-01],
       [2.0206792e-06, 2.3531188e-09, 1.2259523e-04, 9.9303315e-06,
        9.9359916e-09, 9.9986541e-01]], dtype=float32)

In [17]:
y_pred_classes=[np.argmax(i) for i in y_train_pred] 

In [23]:
pd.DataFrame([train_image_names,y_pred_classes]).T.to_csv(r'/Users/vamsi/Downloads/vgg16_without_tunning.csv')